In [ ]:
import pandas as pd

In [7]:
import sys

! pip install --prefix {sys.prefix} scikit-image
! pip install --prefix {sys.prefix} skimage

  Using cached scikit_image-0.16.2-cp37-cp37m-manylinux1_x86_64.whl (26.5 MB)
  Using cached matplotlib-3.1.2-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Using cached Pillow-7.0.0-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Using cached imageio-2.6.1-py3-none-any.whl (3.3 MB)
  Using cached PyWavelets-1.1.1-cp37-cp37m-manylinux1_x86_64.whl (4.4 MB)
  Using cached networkx-2.4-py3-none-any.whl (1.6 MB)
  Using cached pyparsing-2.4.6-py2.py3-none-any.whl (67 kB)
  Using cached kiwisolver-1.1.0-cp37-cp37m-manylinux1_x86_64.whl (90 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Using cached skimage-0.0.tar.gz (757 bytes)
    ERROR: Command errored out with exit status 1:
     command: /home/ec2-user/anaconda3/bin/python -c 'import sys, setupto

In [8]:
import boto3
import numpy as np
import time
import sagemaker

from skimage import io
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.mxnet import MXNet
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner
from sagemaker.mxnet import MXNetModel

ModuleNotFoundError: No module named 'skimage'

# Creating a model using MXNet

## Configure hyperparameters

In [ ]:
# Number of output classes
num_classes = 2

# Batch size for training
mini_batch_size =  128

# Max epochs for training
epochs = 2

# Learning rate
learning_rate = 0.01

## Create a unique job name 

In [ ]:
job_name_prefix = 'breast-cancer-detection'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Specify the input paths for the job

In [ ]:
bucket = 'md-ml-labs-bucket'
input_prefix = 'breast-cancer-detection/input/recordio'
input_train = 's3://{}/{}/train/'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test/'.format(bucket, input_prefix)

## Specify the output path for the job

In [ ]:
output_prefix = 'breast-cancer-detection/output'
output_path = 's3://{}/{}/'.format(bucket, output_prefix)

## Configure training instances

In [ ]:
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 50

## Get the execution role

In [ ]:
role = get_execution_role()

## Configure train timeout

In [ ]:
train_timeout = 360000

## Specify the path to the training script

In [ ]:
training_script_path = 'mxnetScript.py'

## Create a sagemaker.MXNet estimator

In [ ]:
estimator = MXNet(entry_point=training_script_path,
                  role=role,
                  train_instance_count=instance_count,
                  train_instance_type=instance_type,
                  train_volume_size=volume_size_gb,
                  train_max_run=train_timeout,
                  output_path=output_path,
                  framework_version='1.3.0',
                  py_version = 'py3',
                  hyperparameters = {
                    'num-classes': num_classes,
                    'mini-batch-size': mini_batch_size,
                    'epochs': epochs,
                    'learning-rate': learning_rate
                  },
                  metric_definitions = [
                     {
                         'Name': 'Validation-accuracy',
                         'Regex': 'validation: accuracy=([0-9\\.]+)'
                     }
                  ])

# Create a training job

In [ ]:
estimator.fit({
    'train': input_train,
    'test': input_test
}, job_name = job_name)

# Creating a tuning job

## Defining tuning configuration

In [ ]:
hyperparameter_ranges = {
    'learning-rate': ContinuousParameter(0.001, 1.0)
}
objective_metric_name = 'Validation-accuracy'
objective_type = 'Maximize'

max_jobs=2
max_parallel_jobs=2

## Create a unique job name

In [ ]:
job_name_prefix = 'bcd-tuning'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Creating a hyperparameter tuner

In [ ]:
tuner = HyperparameterTuner(estimator=estimator, 
                            objective_metric_name=objective_metric_name, 
                            hyperparameter_ranges=hyperparameter_ranges,
                            objective_type=objective_type, 
                            max_jobs=max_jobs, 
                            max_parallel_jobs=max_parallel_jobs,
                            metric_definitions = [
                                 {
                                     'Name': 'Validation-accuracy',
                                     'Regex': 'validation: accuracy=([0-9\\.]+)'
                                 }
                            ])

In [ ]:
tuner.fit({
    'train': input_train,
    'test': input_test
}, job_name = job_name)
tuner.wait()

# Deploying the best model found by the tuning job

## Get the execution role

In [ ]:
role = get_execution_role()

## Configure hosting instances

In [ ]:
instance_count = 1
instance_type = 'ml.m4.xlarge'

## Specify the path to the training script

In [ ]:
training_script_path = 'mxnetScript.py'

## Create a unique model name

In [ ]:
model_name_prefix = 'bcd-image-classification-mxnet'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = model_name_prefix + timestamp

## Create a MXNetModel object

In [ ]:
model_artifacts_s3_path = 's3://sagemaker-data-jv/breast-cancer-detection/output/bcd-tuning-2019-06-17-13-27-04-002-4dd9bb26/output/model.tar.gz'
model = MXNetModel(
    name=model_name,
    model_data=model_artifacts_s3_path,
    role=role,
    py_version='py3',
    entry_point=training_script_path
)

## Create a unique endpoint name

In [ ]:
endpoint_name_prefix = 'breast-cancer-detection-ep'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = endpoint_name_prefix + timestamp

## Create a model, an endpoint configuration and an endpoint

In [ ]:
predictor = model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=instance_count,
    instance_type=instance_type
)

# Testing the deployed model

In [ ]:
def read_image(filename):
    image = io.imread(filename)
    image = np.array(image).transpose(2, 0, 1)
    image = np.expand_dims(image, axis=0)
    return image

In [ ]:
def predict_breast_cancer(filename):
    image = read_image(filename)
    predicted_class = predictor.predict(image)
    if predicted_class == 0:
        print('Breast cancer not detected')
    else:
        print('Breast cancer detected')

In [ ]:
predict_breast_cancer('images/0/8975_idx5_x2851_y1201_class0.png')

In [ ]:
predict_breast_cancer('images/1/10253_idx5_x551_y651_class1.png')

# Deleting endpoint

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)